In [10]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ximo/GitHub/skforecast


In [ ]:
import re
import pytest
import numpy as np
import pandas as pd
from skforecast.direct import ForecasterDirect
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

# Fixtures
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import y
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog
# from skforecast.direct.tests.tests_forecaster_direct.fixtures_forecaster_direct import exog_predict
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series as series_fixtures
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import series
from skforecast.direct.tests.tests_forecaster_direct_multivariate.fixtures_forecaster_direct_multivariate import exog_predict
series_2 = pd.DataFrame({'l1': np.arange(10), 'l2': np.arange(10, 20)})


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.exceptions import NotFittedError

In [40]:

import re
import pytest
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import norm

from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.recursive import ForecasterRecursive
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import backtesting_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold
from skforecast.preprocessing import RollingFeatures

In [45]:
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection.tests.fixtures_model_selection_multiseries import custom_metric
from skforecast.recursive.tests.tests_forecaster_recursive_multiseries.fixtures_forecaster_recursive_multiseries import expected_df_to_long_format
series_dict = joblib.load('/home/ximo/GitHub/skforecast/skforecast/model_selection/tests/fixture_sample_multi_series.joblib')
exog_dict = joblib.load('/home/ximo/GitHub/skforecast/skforecast/model_selection/tests/fixture_sample_multi_series_exog.joblib')
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}
series_with_nans = series.copy()
series_with_nans.iloc[:10, series_with_nans.columns.get_loc('l2')] = np.nan

In [67]:
import pytest
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect
from skforecast.model_selection._validation import _backtesting_forecaster
from skforecast.model_selection._split import TimeSeriesFold

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y
from skforecast.model_selection.tests.fixtures_model_selection import exog
from skforecast.model_selection.tests.fixtures_model_selection import out_sample_residuals

In [74]:
expected_metric = pd.DataFrame(
    {'mean_squared_error': [0.06099110404144631], 
        'mean_absolute_scaled_error': [0.791393]}
)
expected_predictions = pd.DataFrame(
    data = np.array([[0.55616986, 0.15288789, 0.89198752],
                        [0.48751797, 0.14866438, 0.83169303],
                        [0.57764391, 0.17436194, 0.91346157],
                        [0.51298667, 0.17413308, 0.85716173],
                        [0.47430051, 0.0796644 , 0.82587748],
                        [0.49192271, 0.14609696, 0.95959395],
                        [0.52213783, 0.12750172, 0.8737148 ],
                        [0.54492575, 0.1991    , 1.012597  ],
                        [0.52501537, 0.13641764, 0.86685356],
                        [0.4680474 , 0.08515461, 0.81792677],
                        [0.51059498, 0.12199725, 0.85243317],
                        [0.53067132, 0.14777853, 0.88055069],
                        [0.4430938 , 0.0509291 , 0.69854202],
                        [0.49911716, 0.1231365 , 0.8711497 ],
                        [0.44546347, 0.05329877, 0.70091169],
                        [0.46530749, 0.08932683, 0.83734003],
                        [0.46901878, 0.08173407, 0.82098555],
                        [0.55371362, 0.14618224, 0.98199137],
                        [0.60759064, 0.22030593, 0.9595574 ],
                        [0.50415336, 0.09662198, 0.93243111]]),
    columns = ['pred', 'lower_bound', 'upper_bound'],
    index = pd.RangeIndex(start=30, stop=50, step=1)
)

forecaster = ForecasterDirect(
                    regressor = Ridge(random_state=123), 
                    lags      = 3,
                    steps     = 8
                )

n_backtest = 20
y_train = y[:-n_backtest]
cv = TimeSeriesFold(
        steps                 = 2,
        initial_train_size    = len(y_train),
        window_size           = None,
        differentiation       = None,
        refit                 = 2,
        fixed_train_size      = False,
        gap                   = 0,
        skip_folds            = None,
        allow_incomplete_fold = True,
        return_all_indexes    = False,
    )
metric, backtest_predictions = _backtesting_forecaster(
                                    forecaster              = forecaster,
                                    y                       = y,
                                    exog                    = exog,
                                    cv                      = cv,
                                    metric                  = ['mean_squared_error', 'mean_absolute_scaled_error'],
                                    interval                = [5, 95],
                                    interval_method         = 'bootstrapping',
                                    n_boot                  = 500,
                                    random_state            = 123,
                                    use_in_sample_residuals = True,
                                    use_binned_residuals    = False,
                                    verbose                 = False,
                                    n_jobs                  = 1
                                )
backtest_predictions.to_numpy()

  0%|          | 0/10 [00:00<?, ?it/s]

array([[0.55616986, 0.20571296, 0.92594217],
       [0.48751797, 0.12416328, 0.85598316],
       [0.57764391, 0.22718701, 0.94741621],
       [0.51298667, 0.14963198, 0.88145186],
       [0.47430051, 0.1506004 , 0.8143182 ],
       [0.49192271, 0.16822259, 0.8319404 ],
       [0.52213783, 0.19843771, 0.86215552],
       [0.54492575, 0.22122564, 0.88494344],
       [0.52501537, 0.20809452, 0.89438955],
       [0.4680474 , 0.1611217 , 0.78353678],
       [0.51059498, 0.19367413, 0.87996916],
       [0.53067132, 0.22374562, 0.84616071],
       [0.4430938 , 0.09119872, 0.80766626],
       [0.49911716, 0.14722208, 0.8776036 ],
       [0.44546347, 0.09356839, 0.81003593],
       [0.46530749, 0.11341241, 0.84379393],
       [0.46901878, 0.10970301, 0.8566689 ],
       [0.55371362, 0.21839276, 0.96008583],
       [0.60759064, 0.24827486, 0.99524075],
       [0.50415336, 0.16883251, 0.91052558]])

In [ ]:
array([[0.55616986, 0.20571296, 0.92594217],
       [0.48751797, 0.12416328, 0.85598316],
       [0.57764391, 0.22718701, 0.94741621],
       [0.51298667, 0.14963198, 0.88145186],
       [0.47430051, 0.1506004 , 0.8143182 ],
       [0.49192271, 0.16822259, 0.8319404 ],
       [0.52213783, 0.19843771, 0.86215552],
       [0.54492575, 0.22122564, 0.88494344],
       [0.52501537, 0.20809452, 0.89438955],
       [0.4680474 , 0.1611217 , 0.78353678],
       [0.51059498, 0.19367413, 0.87996916],
       [0.53067132, 0.22374562, 0.84616071],
       [0.4430938 , 0.09119872, 0.80766626],
       [0.49911716, 0.14722208, 0.8776036 ],
       [0.44546347, 0.09356839, 0.81003593],
       [0.46530749, 0.11341241, 0.84379393],
       [0.46901878, 0.10970301, 0.8566689 ],
       [0.55371362, 0.21839276, 0.96008583],
       [0.60759064, 0.24827486, 0.99524075],
       [0.50415336, 0.16883251, 0.91052558]])